In [1]:
import pandas as pd
import numpy as np
from env import get_db_url, user, password, host
from pydataset import data

## Exercises 1

#### 3. Create a function named get_db_url. It should accept a username, hostname, password, and database name and return a url connection string formatted like in the example at the start of this lesson.

In [2]:
# the following function is defined inside env.py

# def get_db_url(database, user=user, password=password, host=host):
#     url = f'mysql+pymysql://{user}:{password}@{host}/{database}'
#     return url

#### 4. Use your function to obtain a connection to the employees database.


In [4]:
url = get_db_url('employees')

#### 5. Successfully run a query

In [5]:
sql = '''
SELECT * 
  FROM employees 
  LIMIT 10'''
pd.read_sql(sql, url)

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12
5,10006,1953-04-20,Anneke,Preusig,F,1989-06-02
6,10007,1957-05-23,Tzvetan,Zielinski,F,1989-02-10
7,10008,1958-02-19,Saniya,Kalloufi,M,1994-09-15
8,10009,1952-04-19,Sumant,Peac,F,1985-02-18
9,10010,1963-06-01,Duangkaew,Piveteau,F,1989-08-24


##### 5a. Intentionally make a typo in the database url. What kind of error message do you see?

In [ ]:
typo_url = f'myseequill+pymysql://{user}:{password}@{host}/employees'
# sql >> seequill
    
pd.read_sql(sql, typo_url)

# returns NoSuchModuleError

In [ ]:
typo_url2 = f'mysql+pymysql{user}:{password}@{host}/employees'
# missing ://

pd.read_sql(sql, typo_url2)

# returns an ArgumentError

In [ ]:
typo_url3 = f'mysql+pymysql://{user}:{password}@{host}/emplyees'
# typo in db name

pd.read_sql(sql, typo_url3)
# returns operational error

#### 5b. Intentionally make an error in your SQL query. What does the error message look like?

In [ ]:
typo_sql = 'ELECT * FROM employees LIMIT 10'

pd.read_sql(typo_sql, url)

# Returns a ProgrammingError that contains the SQL syntax error message

#### 6. Read the employees and titles tables into two separate DataFrames.

In [7]:
sql = 'SELECT * FROM employees'
employees_df = pd.DataFrame(pd.read_sql(sql, url))

In [8]:
sql = 'SELECT * FROM titles'
titles_df = pd.DataFrame(pd.read_sql(sql, url))

#### 7. How many rows and columns do you have in each DataFrame? Is that what you expected?

In [9]:
employees_df.shape
# yes, as expected

(300024, 6)

In [10]:
titles_df.shape
# yes, as expected

(443308, 4)

#### 8. Display the summary statistics for each DataFrame.


In [11]:
employees_df.describe()

,emp_no
count,300024.000000
mean,253321.763392
std,161828.235540
min,10001.000000
25%,85006.750000
50%,249987.500000
75%,424993.250000
max,499999.000000


In [12]:
titles_df.describe()

,emp_no
count,443308.000000
mean,253075.034430
std,161853.292613
min,10001.000000
25%,84855.750000
50%,249847.500000
75%,424891.250000
max,499999.000000


#### 9. How many unique titles are in the titles DataFrame?


In [13]:
titles_df.title.unique().size

7

#### 10. What is the oldest date in the to_date column?

In [14]:
titles_df.to_date.min()

datetime.date(1985, 3, 1)

#### 11. What is the most recent date in the to_date column?


In [15]:
titles_df.to_date.max()

datetime.date(9999, 1, 1)

In [16]:
# excluding the 'current' date
current = titles_df.to_date.max()

titles_df.to_date[titles_df.to_date != current].max()

datetime.date(2002, 8, 1)

## Exercises 2

#### 1. Copy the users and roles DataFrames from the examples.


In [17]:
users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})

In [18]:
roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})

In [19]:
users

,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


In [20]:
roles

,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


#### 2. What is the result of using a right join on the DataFrames?


In [21]:
users.merge(roles, how='right')

,id,name,role_id
0,1,admin,NaN
1,2,author,NaN
2,3,reviewer,NaN
3,4,commenter,NaN


#### 3. What is the result of using an outer join on the DataFrames?


In [22]:
users.merge(roles, how='outer', left_on='role_id', right_on='id')

,id_x,name_x,role_id,id_y,name_y
0,1.0,bob,1.0,1.0,admin
1,2.0,joe,2.0,2.0,author
2,3.0,sally,3.0,3.0,reviewer
3,4.0,adam,3.0,3.0,reviewer
4,5.0,jane,NaN,NaN,NaN
5,6.0,mike,NaN,NaN,NaN
6,NaN,NaN,NaN,4.0,commenter


#### 4. What happens if you drop the foreign keys from the DataFrames and try to merge them?


In [23]:
new_users = users.drop(columns='role_id')

In [24]:
new_users.merge(roles, how='outer')
# no longer able to join on the proper columns since the foreign key doesn't exist

,id,name
0,1,bob
1,2,joe
2,3,sally
3,4,adam
4,5,jane
5,6,mike
6,1,admin
7,2,author
8,3,reviewer
9,4,commenter


#### 5. Load the mpg dataset from PyDataset.


In [25]:
mpg = data('mpg')

#### 6. Output and read the documentation for the mpg dataset.


In [26]:
data('mpg', show_doc=True)

mpg

PyDataset Documentation (adopted from R Documentation. The displayed examples are in R)

## Fuel economy data from 1999 and 2008 for 38 popular models of car

### Description

This dataset contains a subset of the fuel economy data that the EPA makes
available on http://fueleconomy.gov. It contains only models which had a new
release every year between 1999 and 2008 - this was used as a proxy for the
popularity of the car.

### Usage

    data(mpg)

### Format

A data frame with 234 rows and 11 variables

### Details

  * manufacturer. 

  * model. 

  * displ. engine displacement, in litres 

  * year. 

  * cyl. number of cylinders 

  * trans. type of transmission 

  * drv. f = front-wheel drive, r = rear wheel drive, 4 = 4wd 

  * cty. city miles per gallon 

  * hwy. highway miles per gallon 

  * fl. 

  * class. 




#### 7. How many rows and columns are in the dataset?


In [27]:
mpg.shape

(234, 11)

#### 8. Check out your column names and perform any cleanup you may want on them.


In [28]:
mpg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 234 entries, 1 to 234
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   manufacturer  234 non-null    object 
 1   model         234 non-null    object 
 2   displ         234 non-null    float64
 3   year          234 non-null    int64  
 4   cyl           234 non-null    int64  
 5   trans         234 non-null    object 
 6   drv           234 non-null    object 
 7   cty           234 non-null    int64  
 8   hwy           234 non-null    int64  
 9   fl            234 non-null    object 
 10  class         234 non-null    object 
dtypes: float64(1), int64(4), object(6)
memory usage: 21.9+ KB


#### 9. Display the summary statistics for the dataset.


In [29]:
mpg.describe()

,displ,year,cyl,cty,hwy
count,234.000000,234.000000,234.000000,234.000000,234.000000
mean,3.471795,2003.500000,5.888889,16.858974,23.440171
std,1.291959,4.509646,1.611534,4.255946,5.954643
min,1.600000,1999.000000,4.000000,9.000000,12.000000
25%,2.400000,1999.000000,4.000000,14.000000,18.000000
50%,3.300000,2003.500000,6.000000,17.000000,24.000000
75%,4.600000,2008.000000,8.000000,19.000000,27.000000
max,7.000000,2008.000000,8.000000,35.000000,44.000000


#### 10. How many different manufacturers are there?


In [30]:
mpg.manufacturer.unique().size

15

#### 11. How many different models are there?


In [31]:
mpg.model.unique().size

38

#### 12. Create a column named mileage_difference like you did in the DataFrames exercises; this column should contain the difference between highway and city mileage for each car.


In [32]:
mpg['mileage_difference'] = mpg.hwy - mpg.cty

#### 13. Create a column named average_mileage like you did in the DataFrames exercises; this is the mean of the city and highway mileage.


In [33]:
mpg['average_mileage'] = mpg[['hwy', 'cty']].mean(axis=1)

#### 14. Create a new column on the mpg dataset named is_automatic that holds boolean values denoting whether the car has an automatic transmission.


In [34]:
mpg['is_automatic'] = mpg.trans.str.contains('auto')

#### 15. Using the mpg dataset, find out which which manufacturer has the best miles per gallon on average?


In [35]:
mpg.groupby('manufacturer').average_mileage.mean().idxmax()

'honda'

#### 16. Do automatic or manual cars have better miles per gallon?


In [36]:
mpg.groupby('is_automatic').average_mileage.mean()
# manual cars (represented by False in the is_automatic column), have higher average mpg

is_automatic
False    22.227273
True     19.130573
Name: average_mileage, dtype: float64

# Exercises 3

#### 1. Use your get_db_url function to help you explore the data from the chipotle database.


In [40]:
url = get_db_url('chipotle')

In [41]:
sql = 'SELECT * FROM orders'
orders = pd.read_sql(sql, url)

In [42]:
orders.head()

,id,order_id,quantity,item_name,choice_description,item_price
0,1,1,1,Chips and Fresh Tomato Salsa,nan,$2.39
1,2,1,1,Izze,[Clementine],$3.39
2,3,1,1,Nantucket Nectar,[Apple],$3.39
3,4,1,1,Chips and Tomatillo-Green Chili Salsa,nan,$2.39
4,5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]",$16.98


#### 2. What is the total price for each order?

In [43]:
orders['item_price'] = orders.item_price.str.replace('$', '').astype(float) 

/var/folders/zb/3lg9b5xn3831bhkh23bd5bs00000gn/T/ipykernel_27939/345477983.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  orders['item_price'] = orders.item_price.str.replace('$', '').astype(float)


In [44]:
orders.groupby('order_id').item_price.sum()

order_id
1       11.56
2       16.98
3       12.67
4       21.00
5       13.70
        ...  
1830    23.00
1831    12.90
1832    13.20
1833    23.50
1834    28.75
Name: item_price, Length: 1834, dtype: float64

#### 3. What are the most popular 3 items?


In [45]:
orders.groupby('item_name').quantity.sum().nlargest(n=3, keep='all')

item_name
Chicken Bowl           761
Chicken Burrito        591
Chips and Guacamole    506
Name: quantity, dtype: int64

#### 4. Which item has produced the most revenue?


In [46]:
orders.groupby('item_name').item_price.sum().nlargest(n=1, keep='all')

item_name
Chicken Bowl    7342.73
Name: item_price, dtype: float64

In [47]:
orders.groupby('item_name').item_price.sum().idxmax()

'Chicken Bowl'

#### 5. Join the employees and titles DataFrames together.


In [48]:
emp_titles = employees_df.merge(titles_df).sort_values(by='emp_no')
emp_titles.head()

,emp_no,birth_date,first_name,last_name,gender,hire_date,title,from_date,to_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,Senior Engineer,1986-06-26,9999-01-01
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21,Staff,1996-08-03,9999-01-01
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28,Senior Engineer,1995-12-03,9999-01-01
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Engineer,1986-12-01,1995-12-01
4,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Senior Engineer,1995-12-01,9999-01-01


#### 6. For each title, find the hire date of the employee that was hired most recently with that title.


In [49]:
emp_titles.groupby('title').hire_date.max()

title
Assistant Engineer    1999-12-24
Engineer              2000-01-28
Manager               1992-02-05
Senior Engineer       2000-01-01
Senior Staff          2000-01-13
Staff                 2000-01-12
Technique Leader      1999-12-31
Name: hire_date, dtype: object

#### 7. Write the code necessary to create a cross tabulation of the number of titles by department. (Hint: this will involve a combination of SQL code to pull the necessary data and python/pandas code to perform the manipulations.)


In [51]:
url = get_db_url('employees')

sql = '''SELECT d.dept_name, t.title
           FROM employees e
             JOIN titles t USING(emp_no)
             JOIN dept_emp de USING(emp_no)
             JOIN departments d USING(dept_no)'''

df = pd.read_sql(sql, url)

In [52]:
pd.crosstab(df.title, df.dept_name)

dept_name,Customer Service,Development,Finance,Human Resources,Marketing,Production,Quality Management,Research,Sales
title,,,,,,,,,
Assistant Engineer,298,7769,0,0,0,6445,1831,378,0
Engineer,2362,58135,0,0,0,49649,13852,2986,0
Manager,4,2,2,2,2,4,4,2,2
Senior Engineer,2027,49326,0,0,0,42205,11864,2570,0
Senior Staff,13925,1247,12139,12274,13940,1270,0,11637,36191
Staff,16150,1424,13929,14342,16196,1478,0,13495,41808
Technique Leader,309,7683,0,0,0,6557,1795,393,0
